In [ ]:
# !pip install python-amazon-sp-api
# !pip install python-dotenv

In [1]:
from glob import glob
import pandas as pd
import requests
import json
import urllib.parse
from typing import List
from pprint import pprint
from datetime import datetime, timedelta, date
import os
from dotenv import dotenv_values
# from os import getenv
# from dotenv import load_dotenv
# load_dotenv()

In [2]:
secrets = dotenv_values(".env")


In [3]:
# Getting the LWA access token using the Seller Central App credentials. The token will be valid for 1 hour until it expires.
token_response = requests.post(
    "https://api.amazon.com/auth/o2/token",
    data={
        "grant_type": "refresh_token",
        "refresh_token": secrets['refresh_token'],
        "client_id": secrets["lwa_app_id"],
        "client_secret": secrets["lwa_client_secret"],
    },
)
access_token = token_response.json()["access_token"]

In [ ]:
# North America SP API endpoint (from https://developer-docs.amazon.com/sp-api/docs/sp-api-endpoints)
endpoint = "https://sellingpartnerapi-na.amazon.com"

# US Amazon Marketplace ID (from https://developer-docs.amazon.com/sp-api/docs/marketplace-ids)
marketplace_id = "ATVPDKIKX0DER"

# Downloading orders (from https://developer-docs.amazon.com/sp-api/docs/orders-api-v0-reference#getorders)
# the getOrders operation is a HTTP GET request with query parameters
request_params = {
    "MarketplaceIds": marketplace_id,  # required parameter
    "CreatedAfter": (
        datetime.now() - timedelta(days=30)
    ).isoformat(),  # orders created since 30 days ago, the date needs to be in the ISO format
}

orders = requests.get(
    endpoint
    + "/orders/v0/orders"  # getOrders operation path (from https://developer-docs.amazon.com/sp-api/docs/orders-api-v0-reference#getorders)
    + "?"
    + urllib.parse.urlencode(request_params),  # encode query parameters to the URL
    headers={
        "x-amz-access-token": access_token,  # access token from LWA, every SP API request needs to have this header
    },
)

# pretty print the JSON response
print(json.dumps(orders.json(), indent=2))

In [5]:
for i in orders.json()['payload']:
    print(i)

Orders
NextToken
CreatedBefore


In [8]:
request_params = {
    "marketplaceIds": marketplace_id,  # required parameter
    "includedData":  "attributes,dimensions,identifiers,images,productTypes,salesRanks,summaries,relationships" # can include attributes,dimensions,identifiers,images,productTypes,salesRanks,summaries,relationships,vendorDetails
}

asin = "0735211299"

In [9]:
catalog = requests.get(
    endpoint
    + "/catalog/2022-04-01/items/"  # getOrders operation path (from https://developer-docs.amazon.com/sp-api/docs/orders-api-v0-reference#getorders)
    + asin 
    + "?"
    + urllib.parse.urlencode(request_params),  # encode query parameters to the URL
    headers={
        "x-amz-access-token": access_token,  # access token from LWA, every SP API request needs to have this header
    },
)

print(json.dumps(catalog.json(), indent=2))

{
  "asin": "0735211299",
  "attributes": {
    "binding": [
      {
        "value": "hardcover",
        "marketplace_id": "ATVPDKIKX0DER"
      }
    ],
    "item_weight": [
      {
        "unit": "pounds",
        "value": 1.16,
        "marketplace_id": "ATVPDKIKX0DER"
      }
    ],
    "item_dimensions": [
      {
        "width": {
          "unit": "inches",
          "value": 1.1
        },
        "length": {
          "unit": "inches",
          "value": 6.36
        },
        "height": {
          "unit": "inches",
          "value": 9.3
        },
        "marketplace_id": "ATVPDKIKX0DER"
      }
    ],
    "brand": [
      {
        "language_tag": "en_US",
        "value": "Avery",
        "marketplace_id": "ATVPDKIKX0DER"
      }
    ],
    "subject_code": [
      {
        "source": "unknown",
        "type": "thema_classification",
        "value": "KJU",
        "marketplace_id": "ATVPDKIKX0DER"
      },
      {
        "source": "unknown",
        "type": "thema_